# 합성곱 신경망을 사용한 컴퓨터 비전
* CNN은 대뇌의의 시각 피질 연구에서 시작되고 이미지 인식에 사용됨.
### 합성곱 층
* 합성곱 층의 뉴런은 입력 이미지의 모든 픽셀에 연결되는 것이 아니라 수용장 안에 있는 픽셀에만 연결된다.
* 두번째 층의 뉴런은 첫번째 층의 작운 사각영역 안에 위치한 뉴런에 연결된다.
* 네트워크가 첫번째 은닉층에서는 작은 저수준 특성에 집중하고 그 다음 은닉층에서는 더 고수준 특성으로 조합해 나간다.
* 스트라이드가 있다!.
### 필터
* 뉴런의 가중치는 수용장 크기의 작은 이미지로 표현될 수있다.
  * 필터(합성곱 커널)
* 층에 적용된 하나의 필터는 하나의 특성 맵을 만든다.
---
### 여러가지 특성 맵 쌓기
* 실제 합성곱 층은 여러가지 필터를 가지고 필터마다 하나의 특성 맵을 출력하므로 3D로 표현하는 것이 더 정확하다.
* 하나으 피쳐맵의 뉴런은 같은 파라미터를 공유하고 다른 피쳐맵의 뉴런은 다른 파라미터를 사용한다.
* 합성곱 층이 입력에 여러 필터를 동시에 적용하여 입력의 여러 특성을 감지 가능.
* 입력이미지는 컬러채널로 여러 서브층으로 구성되기도 한다. RGB
* 합성곱 l층의 뉴런은 이전층의 해당 위치의 특성맵으로 연결되어있다.
---
### 텐서플로 구현
* 텐서플로에서 각 입력이미지는 보통 [높이,너비,채널]의 3D텐서로 구성된다.
* 하나의 미니배치는 [미니배치 크기,높이,너비,채널] 형태의 4D텐서로 표현된다.
* 합성곱 층의 가중치는 [fh,fw,fn,fn]의 4D텐서로 표현된다. 편향은[fn]텐서
* load_sample_images() 이미지 받고 사용하기 위해 Pillow사용

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import sys
import matplotlib.pyplot as plt


In [ ]:
from sklearn.datasets import load_sample_image
china=load_sample_image("china.jpg")/255 # 각 컬러의 픽셀 강도는 0~255사이의 크기 이를 255로 나눠 0~1로 바꿈
flower=load_sample_image("flower.jpg")/255 #
images=np.array([china,flower])
batch_size,height,wedth,channels=images.shape

In [ ]:
def crop(images):
  return images[150:220,130:250]
def plot_image(image):
    plt.imshow(image, cmap="gray", interpolation="nearest")
    plt.axis("off")

def plot_color_image(image):
    plt.imshow(image, interpolation="nearest")
    plt.axis("off")

In [ ]:
filters=np.zeros(shape=(7,7,channels,2),dtype=np.float32) #7*7 필터 만듬
filters[:,3,:,0]=1 #가운데 흰 수직선
filters[3,:,:,1]=1 # 가운데 수평선

output=tf.nn.conv2d(images,filters,strides=1,padding="SAME")# 제로패딩과 스트라이드 1
# images는 4D텐서, filters는 4D텐서, padding= "VALID"의 경우 제로패딩 안씀.
plt.imshow(output[1,:,:,0],cmap="gray")
plt.show()

In [ ]:
plot_image(crop(images[0, :, :, 0]))
plt.show()

for feature_map_index, filename in enumerate(["china_vertical", "china_horizontal"]):
    plot_image(crop(output[0, :, :, feature_map_index]))
    plt.show()

* 신경망 conv층
  * 3*3사이즈의 32개 필터와 1스트라이드 세임패딩,렐루함수

In [ ]:
conv=tf.keras.layers.Conv2D(filters=32,kernel_size=3,strides=1,
                         padding="same",activation="relu")

### 메모리 요구 사항
* CNN은 많은 램을 사용한다.
* 5*5필터로 스트라이드1 "same"패딩을 사용해 150*100크기의 특성맵 200개의 합성곱층 생각하자
  * 파라미터 수는(5*5*3+1)*200=15200이다.
  * 200개의 피쳐맵 마다 150*100개의 뉴런을 포함하고 각 뉴런은 5*5*3개의 입력에 대한 가중치 합을 계산해야한다.
  * 즉 2억2500만개의 실수 곱샘이 있다.
  *32비트 부동소수로 표현되면 출력이 ram의 200*150*100*32=9천600만 비트(12MB)를 소모한다. 이는 한 샘플에 대한 것이다.
  * 훈련 배치가 100개의 샘플로 이루어져 있다면 이는 1.2GB의 램을 사용할 것이다.
  * 추론을 위해서는 하나의 층이 점유하는 램은 다음층에서 해소된다.
  * 그러나 훈련시에는 역방향을 위해 모두 보전되어야 한다.
  * 메모리 부족으로 훈련이 실패한다면 미니배치 크기를 줄이거나 스트라이드를 사용해 차원을 줄이거나 층을 제거할 수 있다. 32비트 대신 16비트 부동소수도 가능.
  * 여러 장치에 CNN 분산도 가능.
---
### 풀링 층
* 계산량과 메모리 사용량,파라미터수 줄임(과대적합 해소)를 위한 부표본 생성
* 동일하게 크기,스트라이드,패딩을 지정하지만 가중치가 없다.
* 최대나 평균같은 합산 함수 사용이 전부이다.
* 2*2커널 2 스트라이드를 사용하면 이밎의 높이와 너비는 각 절반이된다.
* 작은 변화에도 일정 수준의 불변성을 만든다. 이동 불변성 가짐.
---
### 텐서플로 구현
*  MaxPool2D,AvgPool2D
* 멕스풀링이 더 성능이 좋다. 정보손실은 더 있지만 큰 특징 유지한다.

In [ ]:
max_pool=tf.keras.layers.MaxPool2D(pool_size=2)

In [ ]:
cropped_images = np.array([crop(image) for image in images], dtype=np.float32)
output = max_pool(cropped_images)

In [ ]:
plt.figure(figsize=(12, 8))
plt.subplot(121)
plt.imshow(cropped_images[0])
plt.subplot(122)
plt.imshow(output[0])

* 케라스는 깊이방향 풀링을 제공하지 않지만 만들 수 있다.
* 깊이방향 풀링은 회전층에 상관 없이 동일 출력을 만든다.

In [ ]:
class DepthMaxPool(tf.keras.layers.Layer):
    def __init__(self, pool_size, strides=None, padding="VALID", **kwargs):
        super().__init__(**kwargs)
        if strides is None:
            strides = pool_size
        self.pool_size = pool_size
        self.strides = strides
        self.padding = padding
    def call(self, inputs):
        return tf.nn.max_pool(inputs,
                              ksize=(1, 1, 1, self.pool_size),
                              strides=(1, 1, 1, self.pool_size),
                              padding=self.padding)

In [ ]:
depth_pool = DepthMaxPool(3)
with tf.device("/cpu:0"): # there is no GPU-kernel yet
    depth_output = depth_pool(cropped_images)
depth_output.shape

In [ ]:
# 케라스 모델의 층으로 사용하고싶을때 사용자 정의(위에) 또는 람다 방식으로!.
depth_pool = tf.keras.layers.Lambda(lambda X: tf.nn.max_pool(
    X, ksize=(1, 1, 1, 3), strides=(1, 1, 1, 3), padding="VALID"))
with tf.device("/cpu:0"): # there is no GPU-kernel yet
    depth_output = depth_pool(cropped_images)
depth_output.shape

* 전역 평균 풀링 층!
  * 각 특성 맵의 평균을 계산한다.
  * 특성 맵 마다 하나의 출력을 뽑는다.(대부분의 정보 손실)
  * 매우 파괴적이지만 출력층에는 유용하다.

In [ ]:

global_avg_pool=tf.keras.layers.GlobalAveragePooling2D()
global_avg_pool(cropped_images)

In [ ]:
output_global_avg2=tf.keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=[1,2]))
output_global_avg2(cropped_images)

### CNN구조
* 전형적인 CNN구조는 합성곱 층을 몇 개 쌓고, 그 다음에 풀링층을 쌓고 또 합성곱 층을 쌓고 풀링쌓고 이런식이다.
* 네트워크를 통과할수록 이미지는 작아지지만 일반적으로 합성곱 층 때문에 점점 깊어진다.
* 합성곱 층에 너무 큰 커널을 사용하는것은 흔한 실수이다.
  * 5*5대신 3*3 두개를 쌓는게 계산량이 적고 더 나은 성능을 낸다.
  * 예외로 첫번째 합성곱 층은 큰 크기의 커널과 2이상의 스트라이드를 사용한다. 정보를 너무 잃지 않고 공간 방향 차원을 줄일 수 있다.
  * CNN 출력층에 다다를수록 필터 개수가 늘어난다. 저수준 특성을 연결해서 고수준 특성을 만들기에 합리저.
  * 필터 개수를 두배로 늘리는 것이 일반적이다.

In [ ]:
from tensorflow import keras
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train, X_valid = X_train_full[:-5000], X_train_full[-5000:]
y_train, y_valid = y_train_full[:-5000], y_train_full[-5000:]

X_mean = X_train.mean(axis=0, keepdims=True)
X_std = X_train.std(axis=0, keepdims=True) + 1e-7
X_train = (X_train - X_mean) / X_std
X_valid = (X_valid - X_mean) / X_std
X_test = (X_test - X_mean) / X_std

X_train = X_train[..., np.newaxis]
X_valid = X_valid[..., np.newaxis]
X_test = X_test[..., np.newaxis]

In [ ]:
from functools import partial

DefaultConv2D = partial(keras.layers.Conv2D,
                        kernel_size=3, activation='relu', padding="SAME")
model=keras.models.Sequential([
                               DefaultConv2D(filters=64,kernel_size=7,input_shape=[28,28,1]),
                               keras.layers.MaxPooling2D(pool_size=2),
                               DefaultConv2D(filters=128),
                               DefaultConv2D(filters=128),
                               keras.layers.MaxPooling2D(pool_size=2),
                               DefaultConv2D(filters=256),
                               DefaultConv2D(filters=256),
                               keras.layers.Flatten(),
                               keras.layers.Dense(128,activation="relu"),
                               keras.layers.Dropout(0.5),
                               keras.layers.Dense(64,activation="relu"),
                               keras.layers.Dropout(0.5),
                               keras.layers.Dense(10,activation="softmax")
])

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",optimizer="nadam",metrics=["accuracy"])
model.fit(X_train,y_train,epochs=5,validation_data=(X_valid,y_valid),batch_size=20)
score=model.evaluate(X_test,y_test)
X_new=X_test[:10]
model.predict(X_new),score

### LeNet-5
* 1998 르쿤, MNist에 사용

In [ ]:
model=keras.models.Sequential([
                               keras.layers.Conv2D(filters=6,kernel_size=5,strides=1,activation="tanh",input_shape=[28,28,1]),
                               keras.layers.AveragePooling2D(pool_size=2,strides=2),
                               keras.layers.Conv2D(filters=16,kernel_size=3,strides=1,activation="tanh"),
                               keras.layers.AveragePooling2D(pool_size=2,strides=2),
                               keras.layers.Conv2D(filters=120,kernel_size=5,strides=1,activation="tanh"),
                               keras.layers.Flatten(),
                               keras.layers.Dense(84,activation="softmax")
])

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",optimizer="adam",metrics=["accuracy"])
model.fit(X_train,y_train,batch_size=20,epochs=5,validation_data=(X_valid,y_valid))
model.evaluate(X_test,y_test)

### AlexNet
* 2012 이미지넷 대회 성능 매우 향승
* 드롭아웃적용.
* 이미지를 랜덤하게 이동하거나 수평으로 뒤집고 조명을 바꾸는 등 데이터 증식.

In [ ]:
model=keras.models.Sequential([
                               keras.layers.Conv2D(filters=96,kernel_size=11,strides=4,activation="relu",input_shape=[227,227,3]),
                               keras.layers.MaxPooling2D(pool_size=3,strides=2),
                               keras.layers.Conv2D(filters=256,kernel_size=5,strides=1,padding="same",activation="relu"),
                               keras.layers.MaxPooling2D(pool_size=3,strides=2),
                               keras.layers.Conv2D(filters=384,kernel_size=3,strides=1,padding="same",activation="relu"),
                               keras.layers.Conv2D(filters=384,kernel_size=3,strides=1,padding="same",activation="relu"),
                               keras.layers.Conv2D(filters=384,kernel_size=3,strides=1,padding="same",activation="relu"),
                               keras.layers.MaxPooling2D(pool_size=3,strides=2),
                               keras.layers.Dense(4096,activation="relu"),
                               keras.layers.Dropout(rate=0.5),
                               keras.layers.Dense(4096,activation="relu"),
                               keras.layers.Dense(1000,activation="softmax")

])

In [ ]:
model.summary()

### GoogleNet
* GoogleNet은 Alexnet보다 10배 적은 파라미터를 가진다.
* 1*1,3*3,5*5층 등을 사용해서 각 연산후 concat해서 하나의 출력을 만든다.
----
### VGGNET
* 2014년 2등,
---
### ResNet
* 잔차 네트워크를 사용해 2015 우승.
* 152개 층의 매우 깊은 CNN
* 적은 파라미터로 깊은 네트워크 훈련하는 트렌드 만듬.
* 스킵연결,숏컷 연결. 어떤층에 주입되는 신호가 상위 층의 출력에 더해진다.
* 출력에 x(입력)을 더하면 h(x) 데신 f(x)=h(x)-x로 학습할 것이다. 이를 잔차학습이라 한다.
* 신경망 초기화시 가중치가 0에 가깝기에 네트워크도 0에 가까운 값을 출력한다.
  * 스킵 연결을 추가하면 이 네트워크는 입력과 같은 값을 출력한다.
  * 초기에 항등 함수를 모델링한다. 목적함수가 항등함수에 가깝기에 훈련속도가 매우 빠르다.
* 스킵 연결 덕에 입력 신호가 전체 네트워크에 손쉽게 영향을 끼친다.
* 잔차 유닛(RU)=Relu,3*3커널,배치정규화 사용.
* 잔차 유닛을 매우 깊게 쌓고 배치정규화함
  * 특성의 크기가 바뀌는 잔차유닛에서는 스트라이드 2이고 출력 특성매의 수가 같은 1*1 합성곱 층으로 입력을 통과시긴다.
* ResNet-34는 34는 34개 층으로 이루어진 ResNet으로 64개의 피쳐맵 출력하는 3개 RU, 128개 맵의 4개 RU,256개 맵의 6개RU, 원본 깊이를 복원하는 256개 피쳐맵의 1*1 합성곱층으로 이루어짐.
*